In [65]:
import gc
import os
import ccxt
import talib
import optuna
import joblib
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from datetime import datetime

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from config import binance_client, prices_collection
from helper import vwap, long_short

load_dotenv()

client = binance_client()
prices = prices_collection()

In [66]:
symbol = 'ETHUSD'
balance = client.get_asset_balance(asset='USD')
quantity = float(balance['free'])
order = client.create_order(symbol=symbol, side='BUY', type='MARKET', quoteOrderQty=quantity)
print(order)

{'symbol': 'ETHUSD', 'orderId': 1276491250, 'orderListId': -1, 'clientOrderId': 'p48SWJ0HOGv5MvJo3Ju05w', 'transactTime': 1671462087002, 'price': '0.0000', 'origQty': '0.09961000', 'executedQty': '0.09961000', 'cummulativeQuoteQty': '118.2490', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1671462087002, 'fills': [{'price': '1187.1200', 'qty': '0.09961000', 'commission': '0.00000000', 'commissionAsset': 'BNB', 'tradeId': 38791082}], 'selfTradePreventionMode': 'NONE'}


In [62]:
df = pd.DataFrame(list(prices.find().sort('date', -1).limit(200)))
df.sort_values('date', inplace=True)

In [63]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']

new_data_query = client.get_historical_klines('ETHUSDT', '5m', str(last_date), limit=1000)

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

df['signal'] = df.apply(long_short, axis=1)

In [64]:
df

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
0,639fdd032588a593b2ef3fee,1178.91,1179.23,1178.78,1178.83,2.58590,NaN,NaN,NaN,NaN,NaN,2022-12-18 13:44:59.999,None
1,639fdd032588a593b2ef3fef,1178.83,1178.83,1178.83,1178.83,0.00000,NaN,NaN,NaN,NaN,NaN,2022-12-18 13:49:59.999,None
2,639fdd032588a593b2ef3ff0,1178.60,1178.60,1178.59,1178.59,0.07030,NaN,NaN,NaN,NaN,NaN,2022-12-18 13:54:59.999,None
3,639fdd032588a593b2ef3ff1,1178.59,1178.59,1178.41,1178.41,1.06751,NaN,NaN,NaN,NaN,NaN,2022-12-18 13:59:59.999,None
4,639fdd032588a593b2ef3ff2,1178.41,1178.41,1177.68,1177.68,1.63421,NaN,NaN,NaN,NaN,NaN,2022-12-18 14:04:59.999,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,63a073a14120102c3f50a43e,1180.21,1181.17,1180.21,1181.15,40.44928,41.370011,-0.484843,1181.989108,1183.776126,1183.088681,2022-12-19 14:19:59.999,Short
296,63a074ec4120102c3f50a441,1181.04,1182.63,1181.03,1182.63,3.21968,47.162585,-0.314732,1182.117286,1183.735192,1183.084117,2022-12-19 14:24:59.999,Short
297,63a076374120102c3f50a444,1182.41,1183.33,1182.29,1183.33,3.37824,49.694162,-0.141376,1182.359829,1183.720721,1183.086564,2022-12-19 14:29:59.999,None
298,63a077824120102c3f50a447,1182.46,1182.75,1179.86,1180.94,25.54926,42.250801,-0.171947,1182.075863,1183.621410,1183.065205,2022-12-19 14:34:59.999,Short


In [ ]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']
last_date = int(round(datetime.timestamp(last_date) * 1000)) + 1

new_data_query = client.get_historical_klines('ETHUSDT', '5m', last_date, limit=1000)

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)
    

df = data.copy()
df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)
#df = df.iloc[-200:][['date', 'close', 'rsi', 'macdhist', 'EMA9', 'EMA55', 'EMA200']]

df['signal'] = df.apply(long_short, axis=1)

In [24]:
data = pd.read_parquet('data/eth_data.parquet')

last_date = int(round(data['date'].iloc[-1].timestamp() * 1000)) + 1

new_data_query = client.get_historical_klines('ETHUSDT', '5m', last_date, limit=1000)

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Open time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    data = pd.concat([data, new_data])
    data.reset_index()

In [25]:
data

,open,high,low,close,vol,date
0,737.81,737.81,737.81,737.81,1.955576e+07,2021-01-01 00:02:24
1,736.38,736.38,736.38,736.38,1.954285e+07,2021-01-01 00:13:38
2,732.09,732.09,732.09,732.09,1.964616e+07,2021-01-01 00:19:44
3,732.21,732.21,732.21,732.21,1.889978e+07,2021-01-01 00:32:22
4,734.75,734.75,734.75,734.75,1.895706e+07,2021-01-01 00:39:39
...,...,...,...,...,...,...
282,1183.83,1184.51,1179.58,1179.58,1.816647e+01,2022-12-19 03:00:00
283,1179.55,1179.55,1176.62,1178.58,9.594960e+00,2022-12-19 03:05:00
284,1179.33,1180.46,1177.55,1179.33,2.938850e+00,2022-12-19 03:10:00
285,1179.33,1180.29,1178.11,1180.29,9.451860e+00,2022-12-19 03:15:00


In [33]:
prices_collection().delete_many({})

In [34]:
prices_collection().insert_many(new_data.to_dict('records'))

In [ ]:
client.get_asset_balance(asset='USD')

In [ ]:
symbol = 'ETHUSD'
if df['signal'].iloc[-1] != df['signal'].iloc[-2]:
    if df['signal'].iloc[-1] == 'Long':
        print('Flipped to long')
        balance = client.get_asset_balance(asset='USD')
        quantity = float(balance['free'])
        order = client.create_market_buy_order(symbol=symbol, quantity=balance)
        print(order)
    if df['signal'].iloc[-1] == 'Short':
        print('Flipped to short')
        balance = client.get_asset_balance(asset='ETH')
        quantity = float(balance['free'])
        order = client.create_market_sell_order(symbol=symbol, quantity=balance)
        print(order)
    if df['signal'].iloc[-1] == 'None':
        print('No change in trend')
        pass
else:
    print('No change in trend')


In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))